<a href="https://colab.research.google.com/github/syjoe02/Krx-Bench/blob/main/KRX_Bench_finance_language_model_instruction_tuning_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KRX-Bench 언어 모델 Instruction Tuning 튜토리얼
- Unsloth와 금융 도메인 합성 데이터셋을 활용하여 효율적으로 모델을 학습시키는 튜토리얼입니다.
- Unsloth는 적은 컴퓨팅 자원으로도 모델을 효율적으로 빠르게 학습시킬 수 있는 방법을 제공하는 라이브러리입니다.

## 튜토리얼 환경 및 세부사항
- 모델: [Meta-Llama-3.1-8B](https://huggingface.co/meta-llama/Meta-Llama-3.1-8B)
- 데이터셋: [amphora/krx-sample-instructions](https://huggingface.co/datasets/amphora/krx-sample-instructions)
- 학습 툴: [Unsloth](https://github.com/unslothai/unsloth)
- 학습 방법 : Supervised Fine-tuning with QLoRA(4bit)
- 학습 컴퓨팅 환경: Google Colab T4

## 1. Unsloth 설치 및 학습 환경 구축

In [ ]:
# Unsloth 설치 및 최신 버전 업그레이드
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install trl bitsandbytes triton xformers peft

# GPU가 softcapping을 지원하는 경우, Flash Attention 2 설치
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

## 2. 모델 및 데이터셋 로드

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None # None으로 지정할 경우 해당 컴퓨팅 유닛에 알맞은 dtype으로 저장됩니다. Tesla T4와 V100의 경우에는 Float16, Ampere+ 이상의 경우에는 Bfloat16으로 설정됩니다.
load_in_4bit = True # 메모리 사용량을 줄이기 위해서는 4bit 양자화를 사용하실 것을 권장합니다.

# 모델 및 토크나이저 선언
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # gated model을 사용할 경우 허깅페이스 토큰을 입력해주시길 바라겠습니다.
)

In [ ]:
# LoRA Adapter 선언
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # 0을 넘는 숫자를 선택하세요. 8, 16, 32, 64, 128이 추천됩니다.
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",], # target module도 적절하게 조정할 수 있습니다.
    lora_alpha = 16,
    lora_dropout = 0, # 어떤 값이든 사용될 수 있지만, 0으로 최적화되어 있습니다.
    bias = "none",    # 어떤 값이든 사용될 수 있지만, "none"으로 최적화되어 있습니다.
    use_gradient_checkpointing = "unsloth", # 매우 긴 context에 대해 True 또는 "unsloth"를 사용하십시오.
    random_state = 42,
    use_rslora = False,
    loftq_config = None
)

## 3. 데이터셋 전처리
데이터셋은 한국어 금융 합성 데이터셋인 `amphora/krx-sample-instruction`를 사용하였고, 데이터셋 템플릿으로는 Alpaca prompt에서 input을 제거한 형태의 프롬프트 템플릿을 사용하였습니다.

In [ ]:
prompt_format = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["prompt"]
    outputs = examples["response"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = prompt_format.format(instruction, output) + EOS_TOKEN # 마지막에 eos token을 추가해줌으로써 모델이 출력을 끝마칠 수 있게 만들어 줍니다.
        texts.append(text)
    return { "formatted_text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("amphora/krx-sample-instructions", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

## 4. 모델 학습

full train을 위해서는 `num_train_epochs=1`로 설정하고, `max_steps`를 설정하지 않으면 됩니다.





In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "formatted_text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # True로 설정하면 짧은 텍스트 데이터에 대해서는 더 빠른 학습 속도로를 보여줍니다.
    args = TrainingArguments( # TrainingArguments는 자신의 학습 환경과 기호에 따라 적절하게 설정하면 됩니다.
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

## 5. 모델 추론

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt_format.format(
        "수요와 공급의 원리에 대해서 설명해줘.", # instruction
        "", # output 생성을 위해 빈 칸으로 설정
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

## 6. 모델 저장 및 업로드

In [ ]:
# LoRA Adapter 저장
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

# Merged model 저장 및 업로드
model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "") # 개인 huggingface token을 사용하여 업로드할 수 있습니다.

## 참고자료

- [Unsloth GitHub](https://github.com/unslothai/unsloth)
- [Unsloth Docs](https://docs.unsloth.ai/)
- [Unsloth Meta-Llama-3.1-8B Finetuning Tutorial](https://colab.research.google.com/drive/1Ys44kVvmeZtnICzWz0xgpRnrIOjZAuxp?usp=sharing)
- [Huggingface PEFT](https://github.com/huggingface/peft)
- [Huggingface TRL](https://github.com/huggingface/trl)
- [LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685)
- [QLoRA: Efficient Finetuning of Quantized LLMs](https://arxiv.org/abs/2305.14314)
- [FlashAttention-2: Faster Attention with Better Parallelism and Work Partitioning](https://arxiv.org/abs/2307.08691)